In [2]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
import arabic_reshaper
from bidi.algorithm import get_display

# تأكد من تثبيت المكتبات:
# pip install arabic-reshaper python-bidi pillow

labels_dict = {
    0: 'مرحبا', 1: 'كيف حالك', 2: 'كم', 3: 'متى', 4: 'مين',
    5: 'ملعب', 6: 'ماذا', 7: 'فين', 8: 'شركة', 9: 'أحبك'
}

# تحميل موديل TFLite
interpreter = tf.lite.Interpreter(model_path="sign_model_optimized.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# تهيئة MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# تشغيل الكاميرا
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    prediction_text = ""

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y])

            if len(landmarks) == 42:
                input_data = np.array([landmarks], dtype=np.float32)

                interpreter.set_tensor(input_details[0]['index'], input_data)
                interpreter.invoke()
                output_data = interpreter.get_tensor(output_details[0]['index'])
                prediction = np.argmax(output_data)

                prediction_text = labels_dict.get(prediction, "غير معروف")

    # عرض النص العربي إذا كان موجود
    if prediction_text != "":
        # تجهيز النص العربي
        reshaped_text = arabic_reshaper.reshape(prediction_text)
        bidi_text = get_display(reshaped_text)

        # تحويل الإطار لـ PIL
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(frame_pil)

        # تحميل الخط - تأكد أن الخط موجود عندك أو غيّره لخط عربي متوفر
        font = ImageFont.truetype("arial.ttf", 40)

        # رسم النص
        draw.text((10, 10), bidi_text, font=font, fill=(0, 255, 0))

        # تحويل الصورة مرة ثانية لـ OpenCV
        frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)

    cv2.imshow("Arabic Sign Language", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
